# Load Guild Tables

In [1]:
import os, sys
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import app_proj.excel_service as XS
print([x for x in dir(XS) if x[0]!='_' and len(x)>8])

['AppendToExcel', 'ReadSheet', 'ResetFile', 'ResetSheet']


In [6]:
DATA_PATH = os.path.join(MODULE_PATH, 'data')
print(DATA_PATH)

C:\Documents\Code\OpenStack\ThievesGuild\codebase\backend\emporium\data


In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Unlockable Thief

In [8]:
inputPath = os.path.join(DATA_PATH, 'GuildData.xlsx')
loadDf = XS.ReadSheet(inputPath, 'UnlockableThief')
loadDf = loadDf.dropna(subset=['Class'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

12

In [9]:
def GetResId(row):
    return f"thief-{row['StartTrait'].split(' ')[0]}-s{row['Stars']}"

loadDf['ResourceId'] = loadDf.apply(lambda x: GetResId(x), axis=1)
loadDf[:3]

,Class,Stars,UnlockThrone,StoreCost,StartTrait,RandomTraits,ResourceId
0,Burglar,1,2,3100,agi 3,0,thief-agi-s1
1,Scoundrel,1,2,3100,cun 3,0,thief-cun-s1
2,Ruffian,1,2,3100,mig 3,0,thief-mig-s1


In [10]:
DB.DeleteTable('emporium', 'UnlockableThief')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'UnlockableThief', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'UnlockableThief')
print('records', len(checkLs))
PD.DataFrame(checkLs)[:4]

table deleted
bulk inserted
records 12


,id,ResourceId,Class,Stars,UnlockThrone,StoreCost,StartTrait,RandomTraits
0,37,thief-agi-s1,Burglar,1,2,3100,agi 3,0
1,38,thief-cun-s1,Scoundrel,1,2,3100,cun 3,0
2,39,thief-mig-s1,Ruffian,1,2,3100,mig 3,0
3,40,thief-agi-s2,Burglar,2,4,6700,agi 4,2


### Unlockable Item

In [11]:
# this erases all previous blueprint unlocks

loadDf = XS.ReadSheet(inputPath, 'UnlockableItem')
loadDf = loadDf.dropna(subset=['Name'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf['Throne'] = loadDf['Throne'].astype('int')
loadDf['Level'] = loadDf['Level'].astype('int')
loadDf['MagicLv'] = loadDf['MagicLv'].astype('int')
loadDf['TotalLv'] = loadDf['TotalLv'].astype('int')
loadDf['StoreCost'] = loadDf['StoreCost'].astype('int')
loadDf.shape[0]

78

In [12]:
def GetResId(itemDx, magic):
    second = ''
    if isinstance(itemDx['Trait'], str):
        second = itemDx['Trait'].split(' ')[0]
    else:
        second = 'cmb' if isinstance(itemDx['Combat'], str) else 'skl'

    magicType = f"-{magic.split(' ')[0]}" if magic else ""
    return f"{itemDx['Slot']}-{second}-t{itemDx['Throne']}-m{itemDx['MagicLv']}{magicType}"

In [13]:
# expand magic item line into 1 item per enchantment

fullItems = []
rawItems = NT.DataframeToDicts(loadDf)

for tm in rawItems:
    
    if tm['MagicLv'] == 0:
        tm['ResourceId'] = GetResId(tm, None)
        tm['Magic'] = None
        res = tm.pop('Enchantments')
        fullItems.append(tm)

    else:
        enchants = tm['Enchantments'].split(', ')
        for ch in enchants:
            copyTm = tm.copy()
            copyTm['ResourceId'] = GetResId(tm, ch)
            copyTm['Magic'] = ch
            res = copyTm.pop('Enchantments')
            fullItems.append(copyTm)

PD.DataFrame(fullItems)[10:15]

,Name,Throne,Level,MagicLv,TotalLv,Slot,StoreCost,Requirement,Trait,Combat,Skill,ResourceId,Magic
10,Gauntlet,1,1,0,1,hands,600,None,None,dmg 1,None,hands-cmb-t1-m0,None
11,Greaves,1,1,0,1,feet,600,None,None,att 1,None,feet-cmb-t1-m0,None
12,Cloak,1,1,0,1,back,600,None,end 1,None,None,back-end-t1-m0,None
13,Twin Daggers,1,2,1,3,weapon,2100,Burglar,agi 1,dmg 1,None,weapon-agi-t1-m1-dmg,dmg 1
14,Twin Daggers,1,2,1,3,weapon,2100,Burglar,agi 1,dmg 1,None,weapon-agi-t1-m1-def,def 1


In [14]:
DB.DeleteTable('emporium', 'UnlockableItem')

DB.InsertBulk('emporium', 'UnlockableItem', fullItems)

checkLs = DB.GetTableDictionary('emporium', 'UnlockableItem')
print('records', len(checkLs))
PD.DataFrame(checkLs)[10:15]

table deleted
bulk inserted
records 201


,id,ResourceId,Name,Throne,Level,MagicLv,TotalLv,Slot,StoreCost,Requirement,Trait,Combat,Skill,Magic
10,614,hands-cmb-t1-m0,Gauntlet,1,1,0,1,hands,600,None,None,dmg 1,None,None
11,615,feet-cmb-t1-m0,Greaves,1,1,0,1,feet,600,None,None,att 1,None,None
12,616,back-end-t1-m0,Cloak,1,1,0,1,back,600,None,end 1,None,None,None
13,617,weapon-agi-t1-m1-dmg,Twin Daggers,1,2,1,3,weapon,2100,Burglar,agi 1,dmg 1,None,dmg 1
14,618,weapon-agi-t1-m1-def,Twin Daggers,1,2,1,3,weapon,2100,Burglar,agi 1,dmg 1,None,def 1


In [15]:
#PD.DataFrame(checkLs)[50:75]

### Thief Level

In [16]:
loadDf = XS.ReadSheet(inputPath, 'ThiefLevel')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf = loadDf.drop('RawPower', axis=1, errors='ignore')
loadDf.shape[0]

12

In [17]:
DB.DeleteTable('emporium', 'ThiefLevel')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'ThiefLevel', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'ThiefLevel')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 12


,id,Level,Experience,Power,TrainPeriod,WoundPeriod,KnockedOutPeriod
3,40,4,1200,50,1 hr,6 min,30 min
4,41,5,2000,72,2 hr,10 min,1 hr
5,42,6,3000,96,4 hr,30 min,2 hr
6,43,7,4200,122,8 hr,1 hr,4 hr


### Castle Room

In [18]:
loadDf = XS.ReadSheet(inputPath, 'CastleRoom')
loadDf = loadDf.dropna(subset=['Name'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

13

In [19]:
DB.DeleteTable('emporium', 'CastleRoom')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'CastleRoom', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'CastleRoom')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 13


,id,Name,UnlockThrone,AllowedPlacement,Description
3,43,Bank,1,"R, L",Increases the max storage amount for gold.
4,44,Warehouse,1,"R, L","Increases the max storage amount for wood, sto..."
5,45,Scholarium,1,"R, L",Train thieves here to upgrade their level.
6,46,Dormitory,2,"R, L",Increases the number of thieves allowed in the...


### Room Upgrade

In [20]:
loadDf = XS.ReadSheet(inputPath, 'RoomUpgrade')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [21]:
DB.DeleteTable('emporium', 'RoomUpgrade')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'RoomUpgrade', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'RoomUpgrade')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Stone_Basic,Stone_Advan,Stone_Throne,Period_Basic,Period_Advan,Period_Throne
3,31,4,2000,2100,2400,12 hr,1 day,2 days
4,32,5,2700,2900,3100,1 day,2 days,3 days
5,33,6,3500,3700,4000,2 days,3 days,4 days
6,34,7,4400,4600,4900,3 days,4 days,5 days


### Throne Room

In [22]:
loadDf = XS.ReadSheet(inputPath, 'ThroneRoom')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [23]:
DB.DeleteTable('emporium', 'ThroneRoom')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'ThroneRoom', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'ThroneRoom')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,MaxRoomCount,MaxRoomLevel,MaxThieves,Throne_Gold,Throne_Stone,MagicSlots
3,31,4,8,4,9,2200,1100,1
4,32,5,9,5,10,2600,1400,2
5,33,6,10,6,11,3000,1700,2
6,34,7,10,7,12,3400,2000,3


### Basic Room

In [24]:
loadDf = XS.ReadSheet(inputPath, 'BasicRoom')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [25]:
DB.DeleteTable('emporium', 'BasicRoom')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'BasicRoom', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'BasicRoom')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Keep_Defenders,Keep_Traps,Bank_Gold,Warehouse_Stone,Scholarium_MaxLevel,Dorm_MaxThieves,Dorm_Recovery,Cartog_Slots,Cartog_Recovery
3,31,4,1,2,5200,1400,7,2,20 min,1,10 min
4,32,5,2,2,6300,1900,8,3,40 min,2,20 min
5,33,6,2,3,8500,2500,9,3,1 hr,2,30 min
6,34,7,3,3,10800,3200,10,4,2 hr,2,1 hr


### Advanced Room

In [26]:
loadDf = XS.ReadSheet(inputPath, 'AdvancedRoom')
loadDf = loadDf.dropna(subset=['Level'])
loadDf = loadDf.reset_index().drop(['index'], axis=1, errors='ignore')
loadDf.shape[0]

9

In [27]:
DB.DeleteTable('emporium', 'AdvancedRoom')

loadLs = NT.DataframeToDicts(loadDf)
DB.InsertBulk('emporium', 'AdvancedRoom', loadLs)

checkLs = DB.GetTableDictionary('emporium', 'AdvancedRoom')
print('records', len(checkLs))
PD.DataFrame(checkLs)[3:7]

table deleted
bulk inserted
records 9


,id,Level,Fence_GoldBonus,Fence_MagicSlots,Workshop_StoneBonus,Workshop_Defense,Jeweler_GemBonus,Jeweler_ExpedSlots,Blacksmith_Period,Artisan_Cost,Artisan_Period
3,31,4,10,2,10,thief +1,2,1,16 hr,1100,12 hr
4,32,5,12,2,12,None,2,1,20 hr,1200,16 hr
5,33,6,14,3,14,trap +2,2,1,1 day,1300,20 hr
6,34,7,16,3,16,None,3,2,2 days,1400,1 day


In [28]:
# check end results 

infoLs = DB.GetTableCounts()
PD.DataFrame(infoLs)

,Module,Table,Count
0,members,User,1
1,emporium,UnlockableThief,12
2,emporium,UnlockableItem,201
3,emporium,ThiefLevel,12
4,emporium,CastleRoom,13
5,emporium,RoomUpgrade,9
6,emporium,ThroneRoom,9
7,emporium,BasicRoom,9
8,emporium,AdvancedRoom,9
9,emporium,Trap,72
